## Добро пожаловать

#### Это решение тестового задания 3 by Валерий Леванов. 

Требуется предоставить модель, которая по тексту с пропущенными пробелами, возвращает места пропущенных пробелов.

### 0. Настройка среды

Для корректной работы всего, кажется достаточно наличие pandas, gensim, numpy (возможно, что-то надо заinstallить), файла с эмбеддингами и собственно .txt с данными.

Вот тут можно скачать архив с эмбеддами (взято из дз шада по эмбедам)  [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

In [50]:
import pandas as pd
import gensim
import numpy as np
from gensim.models import KeyedVectors

### 1. Анализ датасета:

Рассмотрим некоторые текста из датасета:

In [47]:
data = pd.read_csv('data.txt', names=['id', 'text_no_spaces'], usecols=[0, 1], skiprows=1)
data.head(10)

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
5,5,работавМосквеудаленно
6,6,куплютелевизорPhilips
7,7,ищугрузчиковдляпереезда
8,8,ремонтквартирподключ
9,9,куплюноутбукHP


In [48]:
data.tail(10)

,id,text_no_spaces
995,995,Весна-ияопятьидугулять.
996,996,Весна-янемогуусидетьдома.
997,997,Весна-ялюблювесну.
998,998,Очемпоютвмоемдворекошки?
999,999,Нет-нет-нет-нет
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,Весна-выпосмотрите
1004,1004,Весна-гдемояголова?


Как можно заметить, часть данных пришла из каких-то реально возможных пользовательских запросов, а часть - строки песен, фольклор и прочее. Это усложняет работу, так как, очевидно у этих данных разные распределения, и эмпирические правила постановки пробелов отличаются. Это мы заметим позднее.

Посмотрим какие вообще символы кроме букв встречаются:

In [60]:
all_texts = ''.join(data['text_no_spaces'].astype(str))

print('Не буквы:', sorted([char for char in set(all_texts) if not char.isalpha()]))

Не буквы: ['!', "'", '+', '-', '.', '0', '1', '2', '3', '4', '5', '7', ':', '?', '«', '»', '–', '—', '’', '…']


К сожалению, есть всякие символы типа длинных тире и "«", "»", которые сложно обобщить под пробелы, их видимо придется ифать(

### 2. Выбор модели

По виду задача не очень сложная, поэтому даже не очень большая LLM должна отлично справлятся с задачей. 

Но у этого подхода есть очевидные минусы: 

- Даже небольшие модели жрут ресурсы и требуют долгое время на обработку предложения (чего бы нам не хотелось)
- У меня нет ресурсов, чтобы разворачивать LLM локально(
- Очень большая вероятность, что у вас не получится повторить вычисления в ноутбуке, так как могут сломаться hf-токены, настройки среды и так далее(



Поэтому остановимся на более простом способе:

Изучив данные, можно заметить что много пробелов можно ставить на основе простых эвристик, а многие наборы слитных слов можно однозначно парсить на отдельные слова. Поэтому итоговая модель будет выглядеть так:

1. Ставим пробелы по эвристикам (?)
2. Каким-то способом делим остатки по словам (?)
3. Какие-то косметические улучшения (?)

Начнём делать все по порядку:

### 3. Постройка модели

#### 3.1. Эвристики

Большая часть эвристик скорее применима к первой части данных (бизнесовые), перечислим часть из них:

Рассматривать проще всего на парах символов ab, если эвристика применима, то ставим между символами пробел

- Смена языка символов - почти всегда означает пробел
- Если a - строчная, b - заглавная, тоже будем считать пробел
- Один символ цифра, а второй нет, очень часто означает пробел (иногда - нет, например в датасете есть $ \textbf{куплюколонки5.1,мощные}$ - жертвуем этим)
- Знаки пунктуации: .,!? тоже часто означают пробел после них. Но на многие из них очень сложно сделать обобщенные правила, поэтому скинем ответственность за них с эвристик на 2 и 3 пункт модели.

In [69]:
import string
import re

def is_cyrillic(c: str) -> bool:
    return '\u0400' <= c <= '\u04FF'

def is_latin(c: str) -> bool:
    return 'a' <= c.lower() <= 'z'

# Функция применения эвристик, которая возвращает текст с пробелами по слитному тексту
def heuristics(text):

    result = []
    current = text[0]

    for i in range(1, len(text)):
        a, b = text[i - 1], text[i]

        # 1 пунктик
        if (is_cyrillic(a) and is_latin(b)) or (is_latin(a) and is_cyrillic(b)):
            result.append(current)
            current = b
            continue

        # 2 пунктик
        if a.islower() and b.isupper():
            result.append(current)
            current = b
            continue

        # 3 пунктик
        if (a.isdigit() and not b.isdigit()) or (not a.isdigit() and b.isdigit()):
            result.append(current)
            current = b
            continue
        current += b

    result.append(current)
    return result

In [70]:
for text in data['text_no_spaces'][:10].astype(str):
    print(text)
    print(heuristics(text))
    print()

куплюайфон14про
['куплюайфон', '14', 'про']

ищудомвПодмосковье
['ищудомв', 'Подмосковье']

сдаюквартирусмебельюитехникой
['сдаюквартирусмебельюитехникой']

новыйдивандоставканедорого
['новыйдивандоставканедорого']

отдамдаромкошку
['отдамдаромкошку']

работавМосквеудаленно
['работав', 'Москвеудаленно']

куплютелевизорPhilips
['куплютелевизор', 'Philips']

ищугрузчиковдляпереезда
['ищугрузчиковдляпереезда']

ремонтквартирподключ
['ремонтквартирподключ']

куплюноутбукHP
['куплюноутбук', 'HP']



Что-то работает. Переходим к следующему пункту:

#### ВАЖНО:

В ходе экспериментов было замечено, что словарик хорошо работает с цифрами, знаками препинания и английскими символами. Поэтому я принял решение отказаться от эвристик в итоговой модели, но, думаю, пусть они останутся, как артефакты моих усилий(. Причем верятно, некоторые эвристики могут поднять точность, но немного, при этом усложнив код, и возможно что-то сломав

#### 3.2. Кромсаем слитные слова

Тут наша задача состоит в разделении колбасы русских слов на маленькие колбаски. Решать задачу можно многими способами. Посмотрев данные, мне показалось, что в большинстве случаев можно достаточно жадно отделять слова. Поэтому хотелось бы такой алгоритм: каким-то образом поделить колбасу, и если все колбаски есть в словаре, то берем такое разбиение. Причем эмпиречески хочется, чтобы колбаски были подлинее.

Возникает много вопросов:

- Где взять словарь?
- На какое число делить колбасу?
- Как понять что одно разбиение лучше другого

В качестве словаря я решил взять "w2v 300 ru" - большой словарик примерно на 1.5 gb. Из плюсов - там много слов. Из минусов - там слишком много слов. Так-же есть некоторая мера частотности слова - норма эмбеддинга. Это нам пригодится. 

In [2]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

Предлагается следующий алгоритм сегментации:

Проходим дп, которое смотрит назад, которое в i строке содержит лучший скор и разбиение. Скор от слова равен 0, если его нет в словаре, иначе - норме эмбеддинга. Лучший скор предложения - наименьшая сумма скоров разбиений. Было замечено чем реже слово встречается, тем меньше его норма, поэтому мы часто получаем большие слова, так как скор слова "дуб" меньше сумм скоров "д" "у" "б". Из проблем, в словаре есть странные редкие слова, которые часто ломают перевод. Но имеем, что имеем.

In [426]:
def word_score(word, model):
    try:
        vec = model.get_vector(word)
        return np.linalg.norm(vec)
    except KeyError:
        return 0

def segment(text, model, max_word_len=20):
    n = len(text)
    dp = [(+float('inf'), [])] * (n + 1)
    dp[0] = (0, [])
    exceptions = ["кварти", "еты", "стобой", "дуя", "тро", "нау", "сомной", "наме", "ив", "ия", "апо", "ной", "нес", "ая", "ана", "акс", "рых"]

    for i in range(1, n + 1):
        current_best_score = +float('inf')
        current_best_split = None
        for j in range(max(0, i - max_word_len), i):
            word = text[j:i]
            score = word_score(word, model)
            if score == 0 or word in exceptions:
                # если слова нет в словарике, пропускаем его
                continue
            cur_score = dp[j][0] + score
            if cur_score < current_best_score:
                current_best_score = cur_score
                current_best_split = dp[j][1] + [word]
        dp[i] = (current_best_score, current_best_split)

    # лучший сплит для всего текста
    return dp[n][1]


Посмотрим, что получилось

In [427]:
for text in data['text_no_spaces'].astype(str).sample(5, random_state=228):
    print(text)
    print(segment(text, ru_emb))
    print()

ищусобакухаски
['ищу', 'собаку', 'хаски']

Выйдунаулицу-солнцанема
['Выйду', 'на', 'улицу', '-', 'солнца', 'нема']

хочузаказатьпиццу
['хочу', 'заказать', 'пиццу']

Втелевизореты
['В', 'телевизоре', 'ты']

куплютелевизорSamsung
['куплю', 'телевизор', 'Samsung']



Можно заметить, что у модели проблема с предлогами. Это понятно, ведь у них большие скоры, и модели хочется слепить их со смежными буквами(тут мы и видим проблему, что есть странные ключи в словаре) 

### 3.3. Допил модели

Пока вот что получается:

In [428]:
for text in data['text_no_spaces'].astype(str).sample(5):
    print(text)
    print(segment(text, ru_emb))
    print()

сдамкомнатусремонтом
['сдам', 'комна', 'тус', 'ремонтом']

Япродолжаюзанимбежать
['Я', 'продолжаю', 'заним', 'бежать']

Позимнемусаду
['По', 'зимнему', 'саду']

Поверьмне
['Поверь', 'мне']

Потёмнымулицамлетит«ночнойдозор».
['По', 'тёмным', 'улицам', 'летит', '«', 'ночной', 'дозор', '»', '.']



1. Видим, что есть всякие куски, которые есть в словаре, но в жизни они не встречаются. Посчитаем самые частые такие куски а засунем их в исключения. Понятно нет смысла делать очень много исключений, так как точности дает немного, но на каждой итерации смотреть лежит ли слово в исключениях - долго.

In [390]:
from collections import Counter

all__words = []
for text in data['text_no_spaces'].astype(str):
    all__words.extend(segment(text, ru_emb))

words = Counter(all__words)

for word, count in words.most_common():
    print(f"{word}: {count}")

ищу: 132
куплю: 118
.: 62
в: 53
сдам: 43
новый: 42
и: 38
Я: 36
И: 33
В: 22
для: 21
новая: 21
мне: 20
А: 19
апо: 18
я: 17
Но: 17
доставка: 16
телевизор: 16
бу: 16
квартиру: 15
тебя: 15
ноутбук: 14
сдаю: 13
срочно: 13
работу: 13
Ты: 13
не: 12
диван: 11
репетитор: 11
комнату: 11
студию: 11
Мне: 11
собаку: 10
без: 10
на: 10
хочу: 10
!: 10
все: 10
-: 10
Весна: 10
кварти: 9
недорого: 9
Samsung: 9
велосипед: 9
меня: 9
Хочешь: 9
телефон: 8
холодильник: 8
так: 8
?: 8
его: 8
под: 7
пылесос: 7
продаю: 7
только: 7
На: 7
есть: 7
из: 7
По: 7
Ну: 7
айфон: 6
ремонт: 6
продам: 6
шкаф: 6
по: 6
центре: 6
врача: 6
LG: 6
дачу: 6
учителя: 6
с: 6
iPhone: 6
Все: 6
Что: 6
О: 6
еты: 6
С: 6
': 6
отдам: 5
мастер: 5
ремонту: 5
Мы: 5
стобой: 5
сам: 5
Только: 5
за: 5
она: 5
сердце: 5
мир: 5
нес: 5
мое: 5
–: 5
дуя: 5
дом: 4
рус: 4
тро: 4
гитару: 4
длительный: 4
срок: 4
машину: 4
куртка: 4
старый: 4
монитор: 4
стол: 4
гараж: 4
посуточно: 4
склад: 4
ремонтом: 4
Москва: 4
купить: 4
нау: 4
лице: 4
Ая: 4
Не: 4
сомной: 4
ж

Хочу исключить: кварти, еты, стобой, дуя, тро, нау, сомной, наме, ив, ия, апо. 

#### Если перезапустить ячейку, очевидно, что эти слова не будут в словаре

2. Эмпирески получено, что возможно если предложение начинается с заглавной, то можно сделать его маленьким и потом уже разбивать, сохранив разбивку для изначального:
3. Знаки препинания! Часть надо соединить с левыми словами, часть с правыми. Также части слов, которые содержат дефи и что-то еще, надо привратить в тире

In [429]:
def final_split(text):
    try:
        segmented = []
        if len(text) > 0 and text[0].isupper():
            copy = text[0].lower() + text[1:]
            segmented = segment(copy, ru_emb)
            if segmented:
                segmented[0] = segmented[0][0].upper() + segmented[0][1:]
        else:
            segmented = segment(text, ru_emb)

        if not segmented:
            return [text]
        
        # Соединим знаки препинания с левыми словами
        for i in range(1, len(segmented)):
            part = segmented[i]
            if len(part) == 1 and part in ["!", "?", ",", ".", ":", ";", "»", "…"]:
                segmented[i - 1] += part
                segmented[i] = ""

        segmented = [part for part in segmented if part]

        # Соединим знаки препинания с правыми словами
        for i in range(len(segmented) - 1):
            part = segmented[i]
            if len(part) == 1 and part in ["«"]:
                segmented[i + 1] = part + segmented[i + 1]
                segmented[i] = ""

        segmented = [part for part in segmented if part]

        # Соединим ковычки с левыми и правыми словами
        for i in range(len(segmented) - 1):
            part = segmented[i]
            if len(part) == 1 and part in ["'", '"']:
                segmented[i + 1] = part + segmented[i + 1]
                segmented[i] = ""
                break

        segmented = [part for part in segmented if part]

        for i in range(1, len(segmented)):
            part = segmented[i]
            if len(part) == 1 and part in ["'", '"']:
                segmented[i - 1] += part
                segmented[i] = ""
                break

        segmented = [part for part in segmented if part]

        # Если часть слова содержит дефис, и буквы, то заменим дефис на тире
        for i in range(len(segmented)):
            part = segmented[i]
            if part[0] == "-" and len(part) > 1:
                segmented[i] = "-"
                segmented.insert(i, "-")
                segmented[i + 1] = part[1:]
            if part[-1] == "-" and len(part) > 1:
                segmented[i] = part[:-1]
                segmented.insert(i+1, "-")

        return segmented

    except Exception as e:
        return [text] 
    

In [430]:
print(final_split("Онаскажет:'А,вообще,Володька,хреново!'"))

['Она', 'скажет:', "'А,", 'вообще,', 'Володька,', "хреново!'"]


Получилось прикольно, теперь переходим к самому сложному:

#### 4. Получение ответа.

In [431]:
def get_answer(text):
    parts = final_split(text)
    spaces_positions = []
    total_length = 0
    for part in parts[:-1]:
        total_length += len(part)
        spaces_positions.append(total_length)
    return spaces_positions

ans_df = pd.DataFrame()
ans_df['id'] = data['id']
ans_df['predicted_positions'] = data['text_no_spaces'].astype(str).apply(get_answer)
ans_df.to_csv('submission.csv', index=False, header=True)

ans_df.head(10)

,id,predicted_positions
0,0,"[5, 10, 12]"
1,1,"[3, 6, 7]"
2,2,"[4, 12, 15, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"
5,5,"[4, 7, 13]"
6,6,"[5, 14]"
7,7,"[3, 12, 15]"
8,8,"[6, 13, 16]"
9,9,"[5, 12]"


Получили: "Your Mean F1 = 79.068%" - для такого слабого и быстрого метода - норм. 

Можем снова посмотреть на части:

In [425]:
all__words = []
for text in data['text_no_spaces'].astype(str):
    all__words.extend(final_split(text))

words = Counter(all__words)

for word, count in words.most_common():
    print(f"{word}: {count}")

ищу: 132
куплю: 118
в: 52
сдам: 43
новый: 42
И: 40
и: 38
Я: 38
-: 32
я: 28
по: 26
квартиру: 22
для: 22
новая: 21
на: 21
А: 20
В: 17
доставка: 16
бу: 16
Но: 16
мне: 16
тебя: 15
телевизор: 14
ноутбук: 14
сдаю: 13
срочно: 13
работу: 13
не: 12
диван: 11
репетитора: 11
комнату: 11
студию: 11
Мне: 11
собаку: 10
без: 10
с: 10
хочу: 10
все: 10
Весна: 10
По: 10
недорого: 9
Samsung: 9
велосипед: 9
Ты: 9
Хочешь: 9
телефон: 8
холодильник: 8
так: 8
меня: 8
его: 8
Ну: 8
под: 7
пылесос: 7
продаю: 7
улице: 7
только: 7
есть: 7
ной: 7
из: 7
за: 7
она: 7
айфон: 6
ремонт: 6
продам: 6
шкаф: 6
центре: 6
врача: 6
LG: 6
дачу: 6
учителя: 6
iPhone: 6
На: 6
Все: 6
Что: 6
отдам: 5
у: 5
мастера: 5
ремонту: 5
Мы: 5
сам: 5
Он: 5
сердце: 5
мир: 5
С: 5
нес: 5
мое: 5
–: 5
ты: 5
дом: 4
метро: 4
гитару: 4
длительный: 4
срок: 4
машину: 4
куртка: 4
старый: 4
монитор: 4
стол: 4
гараж: 4
месяц: 4
посуточно: 4
склад: 4
ремонтом: 4
Москва: 4
купить: 4
Ая: 4
Только: 4
сом: 4
как: 4
Есть: 4
миг: 4
он: 4
день: 4
О: 4
один: 4
хоче

Хочу добавить, еще что-то в исключение, часть того что хотя бы 3 и бред например:

- ной - рискованно, ведь есть такое слово, но мне кажется оно вряд ли попадется
- нес
- ая
- ана
- акс
- рых

Теперь получил: Your Mean F1 = 79.414%. Битва за копейки, так улучшать больше не хочу, ведь падает скорость, и по факту мы просто переобучаемся под выборку.

#### 4. Возможные прокачки метода:

можно попробовать бороться, когда какую часть модель делит на очень много кусочков, так как тогда сильно падает метрика по предложению, но придумать как это отлавливать достаточно трудно, из-за того, что такие штуки могут встречатся в стихотворениях, и можем только ушудшить результат на таких данных

#### 5. Итог.

получилось почти выбить 80 среднего f1. наверное это ок, учитывая 0.5 секунд подсчета для датасета. Всем пока.